# ファンダメンタルデータを使ってパイプラインを作成する

このNotebookは、"ResearchNotebooks-Tutorial and Documentation" の１つである"5. Tutorial - Using Fundamental Data" をベースに、今回の勉強会に沿った形で加筆・修正を行っています。このファイルは、デフォルトで全員のResearchフォルダに保存されているはずです。

## はじめに
ここまでにおいて 「パイプラインの作り方」「ファクターの作り方」「フィルター（ユニバース）の作り方」を見てきました。このNotebookでは、今回のテーマである"ファンダメンタル"データへのアクセス方法を説明します。

これらを合わせることによって、いよいよ「ファンダメンタルファクターモデル」を作るための材料を揃えることができます。


In [ ]:
# パイプラインの作成、及びパイプラインの実行に必要なパッケージのインポート
from quantopian.pipeline import Pipeline
from quantopian.research import run_pipeline

# 今回の主役
from quantopian.pipeline.data import Fundamentals
# 株価データ（脇役）
from quantopian.pipeline.data.builtin import USEquityPricing

# ユニバース
from quantopian.pipeline.experimental import QTradableStocksUS

In [ ]:
# We're going to need a date string for some of our queries. Let's grab one.
import datetime

today = datetime.datetime.now().strftime('%Y-%m-%d')
today

## ユニバースにフィルターを追加する（応用編）

02-04で、QTradableStocksUS という組み込みユニバースと、自作フィルターの使い方を説明しましたが、これらを組み合わせることもできます。

* big_market_cap : 直近の時価総額（＝株価 x 発行済み株式数）が 1,000,000,000ドル以上であること
* big_pe : 直近のpeレシオ（= 株価 / 1株利益）が5倍以上

いよいよ財務データが出てきました。Fundamentalsでどのようなデータが入手可能かについては[こちら](https://www.quantopian.com/help/fundamentals)を参照してください。

In [ ]:
# Create pipeline filters on the market cap and p/e ratio
big_market_cap = Fundamentals.market_cap.latest > 1e9
big_pe = Fundamentals.pe_ratio.latest > 5

# QTradableStockUS に対して、big_market_cap と big_pe の条件を追加することが可能!
universe = QTradableStocksUS() & big_market_cap & big_pe

# パイプライン作成
pipe = pipe = Pipeline(
    columns = {
        'pe_ratio' : Fundamentals.pe_ratio.latest,
        'market_cap' : Fundamentals.market_cap.latest,
    },
    # Universeを指定
    screen = universe
)

# Run the pipeline
fundamental_data = run_pipeline(pipe, start_date = today, end_date = today)
fundamental_data.head()

パイプラインの出力はpandas.Dataframeで渡されるので、pandasの機能で分析することが可能です。せっかくなので、ここ数日のニュースに関連した話題でDataframeを操作します。

iPhone売れ行き不調を受けてアップルの株価が下落していることから、時価総額世界一の座はアップル、アマゾン、マイクロソフトの三つ巴の争いとなっていることが話題となっています。さて、今現在の時価総額世界一の企業はどこでしょうか？


In [ ]:
# pandasのsort_valuesを使って、時価総額の大きい順にソート
fundamental_data.sort_values('market_cap',ascending=False).head()

パイプラインのstart_date と end_date を変更することで、株価や出来高同様、時系列データとして取得することも可能です。

In [ ]:
days_ago_100 = (datetime.datetime.now() - datetime.timedelta(days=100)).strftime('%Y-%m-%d')
fundamental_data = run_pipeline(pipe, start_date = days_ago_100, end_date = today)

fundamental_data.head()